## Usage

### Directions
Log in at <https://aiproxy.sanand.workers.dev/> with your IITM email ID to get your AIPROXY_TOKEN.

Then, instead of sending an API request to `https://api.openai.com/`:

- Replace `https://api.openai.com/` with `https://aiproxy.sanand.workers.dev/openai/`
- Replace the OPENAI_API_KEY with the AIPROXY_TOKEN

For example:

```shell
curl -X POST http://aiproxy.sanand.workers.dev/openai/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $AIPROXY_TOKEN" \
  -d '{"model": "gpt-4o-mini", "messages": [{"role": "user", "content": "What is 2 + 2"}]}'

curl -X POST http://aiproxy.sanand.workers.dev/openai/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $AIPROXY_TOKEN" \
  -d '{"model": "text-embedding-3-small", "input": ["king", "queen"]}'
```

AI Proxy only supports these endpoints and models:

- `GET https://aiproxy.sanand.workers.dev/openai/v1/models`
- `POST https://aiproxy.sanand.workers.dev/openai/v1/embeddings`
  - `model: text-embedding-3-small`
- `POST https://aiproxy.sanand.workers.dev/openai/v1/chat/completions`
  - `model: gpt-4o-mini`

It returns a set of additional headers:

1. `cost`: Cost of this request in USD
2. `monthlyCost`: Total costs (in USD) of requests used this month. Monthly limit is $0.5 (resets at midnight UTC on the first of the next month).
3. `monthlyRequests`: Total requests made this month.

## Set AIProxy token

In [ ]:
AIPROXY_TOKEN = ''

## Chat Completions

In [ ]:
import requests, json
sysprompt = ''
usrprompt = ''
headers = {
    "Authorization": f"Bearer {AIPROXY_TOKEN}",
    "Content-Type": "application/json"
}

data = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "system", "content": sysprompt},
        {"role": "user", "content": usrprompt}
    ],
    "temperature": 0
}

response = requests.post(
    "https://aiproxy.sanand.workers.dev/openai/v1/chat/completions",
    headers=headers,
    data=json.dumps(data)
)

## Vector Embeddings (example sentence similarity code)

In [ ]:
import requests
import numpy as np
import itertools
import json

# Read the text file and extract sentences
with open("your_text_file.txt", "r", encoding="utf-8") as f:
    sentences = [line.strip() for line in f.readlines() if line.strip()]

# API endpoint and headers
url = "https://aiproxy.sanand.workers.dev/openai/v1/embeddings"
headers = {
    "Authorization": f"Bearer {AIPROXY_TOKEN}",
    "Content-Type": "application/json"
}

# Prepare API request payload
payload = {
    "model": "text-embedding-3-small",
    "input": sentences,
    "encoding_format": "float"
}

# Send request to OpenAI API
response = requests.post(url, headers=headers, data=json.dumps(payload))

# Parse response
if response.status_code == 200:
    response_data = response.json()
    embeddings = [entry["embedding"] for entry in response_data["data"]]
else:
    print(f"Error: {response.status_code}, {response.text}")
    exit()

# Function to calculate cosine similarity
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Find similar sentence pairs (cosine similarity > threshold)
threshold = 0.85  # Adjust as needed
similar_pairs = []
for (i, j) in itertools.combinations(range(len(sentences)), 2):
    similarity = cosine_similarity(embeddings[i], embeddings[j])
    if similarity > threshold:
        similar_pairs.append((sentences[i], sentences[j], similarity))

# Print similar sentence pairs
for sent1, sent2, score in similar_pairs:
    print(f"Similarity: {score:.2f}\nSentence 1: {sent1}\nSentence 2: {sent2}\n")
